In [ ]:
# Connect Google drive and read the given dataset
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

# Read dataset.csv
file_path = '/content/drive/My Drive/dataset.csv'
df = pd.read_csv(file_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# General preliminary view of the dataset. Note the obvious presence of numerous NaN values.

df.head(10)

,id,target,sample_type,continuous_0,categorical_0,categorical_1,categorical_2,categorical_3,categorical_4,categorical_5,...,continuous_660,continuous_661,continuous_662,continuous_663,continuous_664,continuous_665,continuous_666,categorical_15,categorical_16,continuous_667
0,14537510,good,dev,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,315.000000,315.00,315.00,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,12527950,good,dev,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,155.000000,155.00,155.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,12277478,good,dev,1.0,1.0,1.0,1.0,1.0,2.0,0.0,...,163.125000,162.15,164.10,0.0,0.0,0.0,0.0,1.0,1.0,NaN
3,13362203,good,dev,1.0,1.0,2.0,2.0,2.0,3.0,0.0,...,107.226666,75.00,153.68,0.0,0.0,0.0,0.0,2.0,2.0,0.0
4,14036057,bad,dev,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,157.150000,155.30,158.45,138.0,46.0,0.0,120.0,3.0,1.0,NaN
5,10586675,good,dev,1.0,2.0,3.0,3.0,3.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,0.0
6,9864565,good,dev,1.0,1.0,3.0,3.0,3.0,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
7,15779547,good,test,1.0,2.0,1.0,1.0,1.0,1.0,0.0,...,95.000000,95.00,95.00,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,10467718,good,valid,1.0,1.0,4.0,4.0,4.0,4.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10661638,good,dev,1.0,1.0,5.0,5.0,5.0,5.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,0.0


In [44]:
# Check the dataframe statistics summary overview. Note the significant amount of NaN values.
df.describe()

,id,continuous_0,categorical_0,categorical_1,categorical_2,categorical_3,categorical_4,categorical_5,categorical_6,categorical_7,...,continuous_660,continuous_661,continuous_662,continuous_663,continuous_664,continuous_665,continuous_666,categorical_15,categorical_16,continuous_667
count,3.149800e+04,31498.000000,31498.000000,31498.000000,31498.000000,31498.000000,31498.000000,31498.000000,31498.000000,31498.000000,...,16405.000000,16405.000000,16405.000000,16405.000000,16405.000000,16405.000000,16405.000000,21025.000000,16477.000000,23746.000000
mean,1.254142e+07,0.983777,1.352022,8.705188,8.717823,8.729126,8.936980,0.512890,0.461966,0.535812,...,107.082979,86.617275,128.735768,43.940013,15.567802,4.295324,30.841700,11.140832,7.656430,0.024846
std,2.301706e+06,0.126335,1.002153,6.513199,6.527480,6.529695,6.759147,0.982387,0.931930,1.021099,...,87.298676,89.529727,97.377377,147.545091,51.530766,26.464109,91.330445,23.194343,5.937698,0.155660
min,9.004705e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.035565e+07,1.000000,1.000000,3.000000,3.000000,3.000000,3.000000,0.000000,0.000000,0.000000,...,55.166666,18.000000,75.000000,0.000000,0.000000,0.000000,0.000000,3.000000,2.000000,0.000000
50%,1.230090e+07,1.000000,1.000000,8.000000,8.000000,8.000000,8.000000,0.000000,0.000000,0.000000,...,90.000000,75.000000,106.000000,0.000000,0.000000,0.000000,0.000000,4.000000,6.000000,0.000000
75%,1.447697e+07,1.000000,1.000000,13.000000,13.000000,13.000000,13.000000,0.000000,0.000000,1.000000,...,150.000000,125.000000,162.000000,20.000000,6.666666,0.000000,18.000000,10.000000,13.000000,0.000000
max,1.712175e+07,1.000000,7.000000,26.000000,26.000000,26.000000,26.000000,5.000000,5.000000,5.000000,...,1510.000000,1510.000000,1510.000000,9030.000000,3010.000000,1200.000000,5220.000000,240.000000,23.000000,1.000000


In [45]:
# Calculate the number of columns with more than 30 percent of NaN values

nan_threshold_perc = 30

nan_percentages = df.isna().mean() * 100  # Percentage of NaN
high_nan_cols = nan_percentages[nan_percentages > nan_threshold_perc]
num_high_nan_cols = len(high_nan_cols)

print(f"Number of columns with more than {nan_threshold_perc}% NaN: {num_high_nan_cols}")

# Delete columns with more than 30% of NaN values

df.drop(columns=high_nan_cols.index, inplace=True)

# Assess the cleaned dataset
print("\nDataFrame info after dropping columns:")
print(df.info())

Number of columns with more than 30% NaN: 178

DataFrame info after dropping columns:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31498 entries, 0 to 31497
Columns: 510 entries, id to continuous_667
dtypes: float64(507), int64(1), object(2)
memory usage: 122.6+ MB
None


The 178 columns with more than 30% of NaN values were deleted, which is a good practise of preprocessing.